In [1]:
%cd '..'

/mnt/scratch/v_kulikov/histonedb_Vladimir/CURATED_SET


In [2]:
import inspect
import os
import re
import sys

import pandas as pd
from Bio import Entrez, SeqIO
from curated_set_services import CuratedSet

In [3]:
# Load data from histones.csv
curated_set = CuratedSet()
cs = curated_set

cs.data.shape, cs.data.columns

((562, 16),
 Index(['accession', 'type', 'variant_group', 'variant', 'doublet', 'gi',
        'ncbi_gene_id', 'hgnc_gene_name', 'taxonomy_id', 'organism', 'phylum',
        'class', 'taxonomy_group', 'info', 'references', 'sequence'],
       dtype='object'))

In [4]:
list(cs.has_duplicates())

[]

In [5]:
cs.data[cs.data["accession"] == "P16889.3"]

Empty DataFrame
Columns: [accession, type, variant_group, variant, doublet, gi, ncbi_gene_id, hgnc_gene_name, taxonomy_id, organism, phylum, class, taxonomy_group, info, references, sequence]
Index: []

In [6]:
cs.data.head()

accession type variant_group         variant doublet  \
NP_505463.1        NP_505463.1  H2A          cH2A  cH2A_(Animals)           
EEC09557.1          EEC09557.1  H2A          cH2A  cH2A_(Animals)           
NP_724343.1        NP_724343.1  H2A          cH2A  cH2A_(Animals)           
XP_001119899.1  XP_001119899.1  H2A          cH2A  cH2A_(Animals)           
EDO48405.1          EDO48405.1  H2A          cH2A  cH2A_(Animals)           

                       gi ncbi_gene_id hgnc_gene_name taxonomy_id  \
NP_505463.1      17562014                                    6239   
EEC09557.1      215500063                                    6945   
NP_724343.1      24585673                                    7227   
XP_001119899.1  110764935                                    7460   
EDO48405.1      156227602                                   45351   

                               organism      phylum        class  \
NP_505463.1      Caenorhabditis elegans    Nematoda  Chromadorea   
EEC09557.1            Ixodes scapularis  Arthropoda    Arachnida   
NP_724343.1     Drosophila melanogaster  Arthropoda      Insecta   
XP_001119899.1           Apis mellifera  Arthropoda      Insecta   
EDO48405.1       Nematostella vectensis    Cnidaria     Anthozoa   

               taxonomy_group info            references  \
NP_505463.1                         26989147 22650316(?)   
EEC09557.1                          26989147 22650316(?)   
NP_724343.1                         26989147 22650316(?)   
XP_001119899.1                      26989147 22650316(?)   
EDO48405.1                          26989147 22650316(?)   

                                                         sequence  
NP_505463.1     MSGRGKGGKAKTGGKAKSRSSRAGLQFPVGRLHRILRKGNYAQRVG...  
EEC09557.1      MSGRGKGGKVKGKSKTRSSRAGLQFPVGRIHRLLRKGNYAERVGAG...  
NP_724343.1     MSGRGKGGKVKGKAKSRSNRAGLQFPVGRIHRLLRKGNYAERVGAG...  
XP_001119899.1  MSGRGKGGKAKAKAKSRSNRAGLQFPVGRIHRLLRKGNYAERVGAG...  
EDO48405.1      MSGRGKGKAKGTKSKTRSSRAGLQFPVGRIHRHLRKGNYAERVGAG...

In [7]:
cs.data.shape

(562, 16)

In [8]:
late_h2b_accessions = [
    "P16889.3",
]

In [9]:
df = pd.DataFrame(
    {
        "accession": late_h2b_accessions,
        "type": ["H2B"] * len(late_h2b_accessions),
        "variant_group": ["late_H2B"] * len(late_h2b_accessions),
        "variant": ["late_H2B.3"] * len(late_h2b_accessions),
        "references": ["3697096"] * len(late_h2b_accessions),
    }
)
df.index = df.accession
df.shape, df.columns

((1, 5),
 Index(['accession', 'type', 'variant_group', 'variant', 'references'], dtype='object'))

In [10]:
df.head()

accession type variant_group     variant references
accession                                                    
P16889.3   P16889.3  H2B      late_H2B  late_H2B.3    3697096

In [11]:
cs.data = pd.concat([cs.data, df]).fillna("")
cs.data.shape, list(cs.has_duplicates())

((563, 16), [])

In [12]:
cs.data.tail()

accession type variant_group                    variant  \
NP_001295191.1  NP_001295191.1   H1          H1.8   H1.8_(Homo_sapiens)__???   
NP_005311.1        NP_005311.1   H1          H1.3   H1.3_(Homo_sapiens)__???   
NP_006017.1        NP_006017.1   H1         H1.10  H1.10_(Homo_sapiens)__???   
NP_005310.1        NP_005310.1   H1          H1.2   H1.2_(Homo_sapiens)__???   
P16889.3              P16889.3  H2B      late_H2B                 late_H2B.3   

               doublet gi ncbi_gene_id hgnc_gene_name taxonomy_id  \
NP_001295191.1                132243.0           H1-8        9606   
NP_005311.1                     3007.0           H1-3        9606   
NP_006017.1                     8971.0          H1-10        9606   
NP_005310.1                     3006.0           H1-2        9606   
P16889.3                                                            

                    organism    phylum     class taxonomy_group info  \
NP_001295191.1  Homo sapiens  Chordata  Mammalia       Mammalia        
NP_005311.1     Homo sapiens  Chordata  Mammalia       Mammalia        
NP_006017.1     Homo sapiens  Chordata  Mammalia       Mammalia        
NP_005310.1     Homo sapiens  Chordata  Mammalia       Mammalia        
P16889.3                                                               

               references                                           sequence  
NP_001295191.1   26689747  MAPATAPRRAGEAKGKGPKKPSEAKEDPPNVGKVKKAAKRPAKVQK...  
NP_005311.1      26689747  MSETAPLAPTIPAPAEKTPVKKKAKKAGATAGKRKASGPPVSELIT...  
NP_006017.1      26689747  MSVELEEALPVTTAEGMAKKVTKAGGSAALSPSKKRKNSKKKNQPG...  
NP_005310.1      26689747  MSETAPAAPAAAPPAEKAPVKKKAAKKAGGTPRKASGPPVSELITK...  
P16889.3          3697096

In [13]:
cs.data[cs.data["variant"] == "late_H2B.3"].shape

(1, 16)

In [14]:
cs.update_accession_version()
cs.data = cs.data.set_index(cs.data.accession.values)

/home/l_singh/.conda/envs/histdb_env/lib/python3.8/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(
/home/l_singh/.conda/envs/histdb_env/lib/python3.8/site-packages/Bio/GenBank/__init__.py:1143: BiopythonParserWarning: Dropping bond qualifier in feature location
  warnings.warn(
/mnt/scratch/v_kulikov/histonedb_Vladimir/CURATED_SET/curated_set_services.py:267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [15]:
cs.data[cs.data["variant"] == "late_H2B.3"].shape

(1, 16)

In [16]:
curated_set.update_taxids(blank_data=True)

Fetched taxid from NCBI 7668
 changes to 7668
 changes to Strongylocentrotus purpuratus
 changes to Echinodermata
 changes to Echinoidea


In [17]:
cs.data[cs.data["variant"] == "late_H2B.3"].shape

(1, 16)

In [18]:
cs.data[cs.data["variant"] == "late_H2B.3"]

accession type variant_group     variant doublet gi ncbi_gene_id  \
P16889.3  P16889.3  H2B      late_H2B  late_H2B.3                           

         hgnc_gene_name taxonomy_id                       organism  \
P16889.3                       7668  Strongylocentrotus purpuratus   

                 phylum       class taxonomy_group info references sequence  
P16889.3  Echinodermata  Echinoidea                        3697096

## Updating sequences for late_H2B.3

In [19]:
cs.data[cs.data['sequence'] == '']

accession type variant_group     variant doublet gi ncbi_gene_id  \
P16889.3  P16889.3  H2B      late_H2B  late_H2B.3                           

         hgnc_gene_name taxonomy_id                       organism  \
P16889.3                       7668  Strongylocentrotus purpuratus   

                 phylum       class taxonomy_group info references sequence  
P16889.3  Echinodermata  Echinoidea                        3697096

In [20]:
cs.update_sequence(blank_data=True)

Fetching 1 seqs


/home/l_singh/.conda/envs/histdb_env/lib/python3.8/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


Sequence for P16889.3 changes from  to MPAKAQAAGKKGSKKAKAPKPSGDKKRRRKRKESYGIYIYKVLKQVHPDTGISSRAMSIMNSFVNDVFERIAAEASRLAHYNKKSTITSREVQTAVRLLLPGELAKHAVSEGTKAVTKYTTSK
Sequences updated: 1


In [21]:
cs.data[cs.data["variant"] == "late_H2B.3"]

accession type variant_group     variant doublet gi ncbi_gene_id  \
P16889.3  P16889.3  H2B      late_H2B  late_H2B.3                           

         hgnc_gene_name taxonomy_id                       organism  \
P16889.3                       7668  Strongylocentrotus purpuratus   

                 phylum       class taxonomy_group info references  \
P16889.3  Echinodermata  Echinoidea                        3697096   

                                                   sequence  
P16889.3  MPAKAQAAGKKGSKKAKAPKPSGDKKRRRKRKESYGIYIYKVLKQV...

In [22]:
cs.save()

Added sequence with accession P16889.3
cp histones.csv backups/histones.csv-Mar2324100043
Previous data backuped to backups/histones.csv-Mar2324100043
Results saved to histones.csv
